<a href="https://colab.research.google.com/github/R2D2Beltran/Contador-separando-datos-de-un-vector-y-accionamiento-de-una-unica-vez-por-boton/blob/master/train_with_groundingdino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labeling with Grounding DINO

Check GPU

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Install libraries

In [2]:
cd/content

/content


In [3]:
import os
HOME = os.getcwd()

Download and install Grounding DINO

In [4]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 463 (delta 176), reused 137 (delta 137), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [5]:
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install supervision

/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 10.4 MB/s eta 0:00:00


Download weights

In [6]:
!mkdir {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content/weights


In [7]:
cd /content/GroundingDINO

/content/GroundingDINO


Object detect with Grounding DINO

Load weights

In [8]:
%cd {HOME}/GroundingDINO
from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(f"{HOME}/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
                   f"{HOME}/weights/groundingdino_swint_ogc.pth")

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Object Detect

In [15]:
import os
import supervision as sv

IMAGE_NAME = "/content/data/images/0217013439.jpg"
IMAGE_PATH = os.path.join(HOME, 'data', IMAGE_NAME)

TEXT_PROMPT = "bottle"
BOX_THRESHOLD = 0.7
TEXT_THRESHOLD = 0.25

image_source, image = load_image(IMAGE_NAME)

boxes, logits, phrases = predict(
    model = model,
    image = image,
    caption = TEXT_PROMPT,
    box_threshold = BOX_THRESHOLD,
    text_threshold = TEXT_THRESHOLD,
    device = 'cuda'
    )

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

%matplotlib inline
sv.plot_image(annotated_frame, (16,16))

FileNotFoundError: [Errno 2] No such file or directory: '/data/images/0217013439.jpg'

Labeling with Grounding DINO

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!find /content/data/images -type d -name ".ipynb_checkpoints" -exec rm -r {} +

In [ ]:
# Libraries
import os
from time import time
import cv2
import torch
from PIL import Image
from GroundingDINO.groundingdino.util.inference import load_model, predict, annotate
import GroundingDINO.groundingdino.datasets.transforms as T

In [ ]:
# Device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def read_images_from_folder(folder_path):
    images = []
    clases = []
    lista = os.listdir(folder_path)

    for lis in lista:
        img_path = os.path.join(folder_path, lis)
        img = cv2.imread(img_path)
        images.append(img)
        clases.append(os.path.splitext(lis)[0])

    return images, clases

def save_results(image, boxes, class_id, out_folder):
    # Norm
    xc, yc, an, al = boxes[0][0], boxes[0][1], boxes[0][2], boxes[0][3]

    xc, yc, an, al = max(0, min(1, xc)), max(0, min(1, yc)), max(0, min(1, an)), max(0, min(1, al))

    list_info = [f"{class_id} {xc} {yc} {an} {al}"]

    time_now = str(time()).replace('.', '')

    cv2.imwrite(f"{out_folder}/{time_now}.jpg", image)

    for info in list_info:
        with open(f"{out_folder}/{time_now}.txt", 'a') as f:
            f.write(info)

def main():
    img_folder_path = '/content/data/images'
    out_folder_path = '/content/data/annotations'
    class_id = 1
    save_results_flag = True

    images, classes = read_images_from_folder(img_folder_path)
    num_images = len(images)

    print(f"Imagenes: {num_images}")
    print(f'Nombres: {classes}')

    home = os.getcwd()

    # Config Path
    config_path = os.path.join(home, "/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")

    # CheckPoint Weights
    check_point_path = '/content/weights/groundingdino_swint_ogc.pth'

    # Model
    model = load_model(config_path, check_point_path)

    # Prompt
    text_prompt = 'flexometer'
    box_threshold = 0.40
    text_threshold = 0.25

    for con in range(num_images):
        img = images[con]
        print("------------------//--------------------")
        print(f"Image: {classes[con]}")

        img_copy = img.copy()

        transform = T.Compose([
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        img_source = Image.fromarray(img).convert("RGB")
        img_transform, _ = transform(img_source, None)

        boxes, logits, phrases = predict(
            model=model,
            image=img_transform,
            caption=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            device=DEVICE)

        if len(boxes) != 0:
            if save_results_flag:
                save_results(img_copy, boxes, class_id, out_folder_path)

        annotated_img = annotate(image_source=img, boxes=boxes, logits=logits, phrases=phrases)
        out_frame = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

if __name__ == "__main__":
    main()

Compress annotations

In [ ]:
!zip -r "/content/data/annotations.zip" "/content/data/annotations"

Delete old images

In [ ]:
from shutil import rmtree
rmtree("/content/dataTrain/train/images")

# Train Yolo v8 Model

In [ ]:
!pip install ultralytics

In [ ]:
cd /content/models

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt

Train custom model

In [ ]:
cd /content

In [ ]:
# Libraries
from ultralytics import YOLO

In [ ]:
# Model
model = YOLO('/content/models/yolov8n.pt')

In [ ]:
#Train
model.train(data='/content/dataTrain/data.yaml', epochs = 40, batch = 64, imgsz = 640, device = 'cuda')